In [ ]:
from nltk.corpus import stopwords
import nltk
import re
from gensim.models import Word2Vec
import pandas as pd 
import numpy as np
from sklearn.datasets import load_files
from collections import Counter 
import gc
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
path = "../data/articles1.csv"
News = pd.read_csv(path)
News.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [ ]:
text= News['content']
X=text.values.tolist()

In [ ]:
def majid(X):
    corpus = []
    for i in range(0, len(X)):
        #review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(X[i])) #remove punctuation
        review = re.sub(r'\d+',' ', str(X[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)        
#    return corpus        
    #Tokenizing and Word Count  
    words=[]
    for i in range(len(corpus)):
        words= nltk.word_tokenize(corpus[i])
        #sentences.append(words)
   
    return words

In [ ]:
X = [[el] for el in X] 

from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()
sentences = Parallel(n_jobs=num_cores)(delayed(majid)(i) for i in X)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:

def remove_stopwords(sentences):
        stopwords_list = nltk.corpus.stopwords.words('english')
#        words = sentences.split() 
        clean_words = [word for word in sentences if (word not in stopwords_list)] 
        return clean_words 

def majid2(X):
    sentences = remove_stopwords(X)
    gc.collect()
    return sentences
from joblib import Parallel, delayed
import multiprocessing

    
num_cores = multiprocessing.cpu_count()
sent2 = Parallel(n_jobs=num_cores)(delayed(majid2)(i) for i in sentences)


In [ ]:
count = 0
c = {}
for words in sent2:
  for s in words:
    if s in c:
        c[s] += 1
    else:
        c[s] = 1
    count+=1
    #if (word_counter % 10000)  == 0:
    #    print(word_counter)
d= []

for k,v in c.items():
    if v == 1:
        d.append(k) 
    
print('Corpus Size=', count)        
print('Unique words=', len(d)) 

Corpus Size= 23482677
Unique words= 70018


In [ ]:
model1 = Word2Vec(sent2, min_count=3,size= 300,workers=multiprocessing.cpu_count(), window =1, sg = 0)
print('Done Training')

SizeOfVocab = model1.wv.vocab
print('Size of Vocabulary=',len(SizeOfVocab))
print('Done making the Vocabulary')

#####
model2 = Word2Vec(sent2, min_count=3,size= 300,workers=multiprocessing.cpu_count(), window =1, sg = 1)
print('Done Training')

SizeOfVocab = model2.wv.vocab
print('Size of Vocabulary=',len(SizeOfVocab))
print('Done making the Vocabulary')

Done Training
Size of Vocabulary= 88651
Done making the Vocabulary
Done Training
Size of Vocabulary= 88651
Done making the Vocabulary


In [ ]:
from gensim.models import Word2Vec, KeyedVectors 
path = "../Embeddings/"  
#model1.wv.save_word2vec_format('/content/drive/MyDrive/NLP/W-CBOW-Stop.bin.gz', binary=True)
model1.wv.save_word2vec_format(path + 'W-CBOW-Stop.txt', binary=False)
model1.save('W-CBOW-Stop.bin')
print('Done Saving Model1')
#####
#model2.wv.save_word2vec_format('/content/drive/MyDrive/NLP/W-Skip-Stop.bin.gz', binary=True)
model2.wv.save_word2vec_format(path + 'W-Skip-Stop.txt', binary=False)
model2.save('W-Skip-Stop.bin')
print('Done Saving Model2')

#model.save('model2.bin')

print('Done Saving the Embeddings')

Done Saving Model1
Done Saving Model2
Done Saving the Embeddings
